In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

In [ ]:
ped_h5_list <- list.files(path='cache/', pattern=".h5$", recursive=TRUE)
br1_h5_list <- list.files(path='BR1/', pattern=".h5$", recursive=TRUE)
br2_h5_list <- list.files(path='BR2/', pattern=".h5$", recursive=TRUE)

In [ ]:
length(ped_h5_list)
length(br1_h5_list)
length(br2_h5_list)

# Pediatric Object Generation

## Create Seurat Object

In [ ]:
ped_h5_list <- paste0('cache/',ped_h5_list)

In [ ]:
so_list <- lapply(ped_h5_list, read_h5_seurat)

In [ ]:
so_merge <- Reduce(merge, so_list)

In [ ]:
table(so_merge$pbmc_sample_id)

In [ ]:
so_merge <- SetIdent(so_merge, value = 'pbmc_sample_id')
so_merge <- RenameIdents(so_merge,
                        'PB00127-01' = 'D0','PB00173-01' = 'D0','PB00175-01' = 'D7',
                        'PB00180-01' = 'D7','PB00182-01' = 'D0','PB00192-01' = 'D0',
                        'PB00193-01' = 'D0','PB00809-01' = 'D0','PB00811-01' = 'D0',
                        'PB00813-01' = 'D7','PB00815-01' = 'D7','PB00828-01' = 'D7',
                        'PB00833-01' = 'D0','PB00841-01' = 'D0','PB00842-01' = 'D0',
                        'PB00847-01' = 'D7','PB00852-01' = 'D7','PB00853-01' = 'D7',
                        'PB00879-01' = 'D7','PB00884-01' = 'D0','PB00886-01' = 'D7',
                        'PB00892-01' = 'D0','PB00907-01' = 'D7','PB00910-01' = 'D0',
                        'PB00913-01' = 'D0','PB00914-01' = 'D0','PB00924-01' = 'D7',
                        'PB00925-01' = 'D7','PB00928-01' = 'D0','PB00931-01' = 'D7',
                        'PB00937-01' = 'D7','PB00961-01' = 'D7')
so_merge$visit <- Idents(so_merge)

In [ ]:
table(so_merge$visit)

In [ ]:
so_merge <- SetIdent(so_merge, value = 'visit')
ped_so <- subset(so_merge, idents = 'D0')
# d7 <- subset(so_merge, idents = 'D7')

In [ ]:
table(ped_so$pbmc_sample_id)

In [ ]:
rm(so_list)
rm(so_merge)

## Normalization

In [ ]:
ped_so <- suppressWarnings(SCTransform(ped_so, conserve.memory = TRUE)) %>% RunPCA()

## Label Transfer

In [ ]:
library(SeuratDisk)

In [ ]:
ref <- LoadH5Seurat('pbmc_multimodal.h5seurat')

In [ ]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = ped_so,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
    )
ped_so <- TransferData(anchorset = anchors, reference = ref, query = ped_so,
    refdata = list(
    celltype.l1 = "celltype.l1",
    celltype.l2 = "celltype.l2",
    celltype.l3 = "celltype.l3",
    predicted_ADT = 'ADT'))

# Young Adult Object Generation

## Create Seurat Object

In [ ]:
br1_h5_list <- paste0('BR1/',br1_h5_list)

In [ ]:
so_list <- lapply(br1_h5_list, read_h5_seurat)

In [ ]:
br1_merge <- Reduce(merge, so_list)

## Subset to 16 aliquots / subjects

In [ ]:
table(br1_merge$batch_id)

In [ ]:
table(br1_merge$pbmc_sample_id)

In [ ]:
br1_merge <- SetIdent(br1_merge, value = 'pbmc_sample_id')
kt010 <- subset(br1_merge, idents = c('PB00010-01','PB00010-02'))
table(kt010$batch_id)

Let's take the more recent batch (B078) since both have very similar cell counts.

In [ ]:
br1_merge <- SetIdent(br1_merge, value = 'pbmc_sample_id')
kt023 <- subset(br1_merge, idents = c('PB00023-03','PB00023-05'))
table(kt023$batch_id)

Let's take the more recent batch (B078) since both have very similar cell counts.

In [ ]:
br1_merge <- SetIdent(br1_merge, value = 'pbmc_sample_id')
kt026 <- subset(br1_merge, idents = c('PB00026-03','PB00026-05'))
table(kt026$batch_id)

Let's again use B078 for this.

In [ ]:
br1_merge <- SetIdent(br1_merge, value = 'pbmc_sample_id')
kt030 <- subset(br1_merge, idents = c('PB00030-02','PB00030-03'))
table(kt030$batch_id)

Let's again use B078 for this.

In [ ]:
br1_merge <- SetIdent(br1_merge, value = 'pbmc_sample_id')
kt031 <- subset(br1_merge, idents = c('PB00031-03','PB00031-05'))
table(kt031$batch_id)

Let's again use B078 for this.

In [ ]:
rm(kt010)
rm(kt023)
rm(kt026)
rm(kt030)
rm(kt031)

In [ ]:
br1_merge <- SetIdent(br1_merge, value = 'pbmc_sample_id')
br1_merge <- subset(br1_merge, idents = c('PB00010-01','PB00023-03','PB00026-03','PB00030-02','PB00031-03'), invert = TRUE)
table(br1_merge$pbmc_sample_id)

## Normalization

In [ ]:
br1_merge <- suppressWarnings(SCTransform(br1_merge, conserve.memory = TRUE)) %>% RunPCA()

## Label Transfer

In [ ]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = br1_merge,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
    )
br1_merge <- TransferData(anchorset = anchors, reference = ref, query = br1_merge,
    refdata = list(
    celltype.l1 = "celltype.l1",
    celltype.l2 = "celltype.l2",
    celltype.l3 = "celltype.l3",
    predicted_ADT = 'ADT'))

In [ ]:
br1_merge

# Older Adult Object Generation

## Create Seurat Object

In [ ]:
br2_h5_list <- paste0('BR2/',br2_h5_list)

In [ ]:
so_list <- lapply(br2_h5_list, read_h5_seurat)

In [ ]:
br2_merge <- Reduce(merge, so_list)

## Subset to 16 aliquots / subjects

In [ ]:
table(br2_merge$batch_id)

Will want to drop EXP-00243

In [ ]:
br2_merge <- SetIdent(br2_merge, value = 'batch_id')
br2_merge <- subset(br2_merge, idents = 'EXP-00243', invert = TRUE)

In [ ]:
table(br2_merge$pbmc_sample_id)

In [ ]:
br2_merge <- SetIdent(br2_merge, value = 'pbmc_sample_id')
kt352 <- subset(br2_merge, idents = c('PB00352-01','PB00352-04'))

In [ ]:
table(kt352$batch_id)

Let's take the more recent batch (B078) with higher cell number.

In [ ]:
br2_merge <- SetIdent(br2_merge, value = 'pbmc_sample_id')
kt561 <- subset(br2_merge, idents = c('PB00561-01','PB00561-04'))

In [ ]:
table(kt561$batch_id)

Let's take the batch with a higher cell number (B054)

In [ ]:
rm(kt352)
rm(kt561)

In [ ]:
br2_merge <- SetIdent(br2_merge, value = 'pbmc_sample_id')
br2_merge <- subset(br2_merge, idents = c('PB00352-01','PB00561-04'),invert=TRUE)

In [ ]:
table(br2_merge$batch_id)
table(br2_merge$pbmc_sample_id)

## Normalization

In [ ]:
br2_merge <- suppressWarnings(SCTransform(br2_merge, conserve.memory = TRUE)) %>% RunPCA()

## Label Transfer

In [ ]:
anchors <- FindTransferAnchors(
    reference = ref,
    query = br2_merge,
    normalization.method = "SCT",
    reference.reduction = "spca",
    dims = 1:50
    )
br2_merge <- TransferData(anchorset = anchors, reference = ref, query = br2_merge,
    refdata = list(
    celltype.l1 = "celltype.l1",
    celltype.l2 = "celltype.l2",
    celltype.l3 = "celltype.l3",
    predicted_ADT = 'ADT'))

In [ ]:
br2_merge

# Merge labeled objects

In [ ]:
ped_so$age <- 'Pediatric'
br1_merge$age <- 'Young Adult'
br2_merge$age <- 'Older Adult'

In [ ]:
so_merge <- merge(ped_so, c(br1_merge, br2_merge))

In [ ]:
saveRDS(so_merge, file = 'age_merge.rds')

In [ ]:
so_merge

In [ ]:
# so_merge <- readRDS(file='age_merge.rds')

# T Cell Clustering

In [ ]:
so_merge <- SetIdent(so_merge, value = 'predicted.celltype.l1')
t_cells <- subset(so_merge, idents = c('CD4 T','CD8 T','other T'))

In [ ]:
rm(so_merge)

In [ ]:
t_cells <- suppressWarnings(SCTransform(t_cells, conserve.memory = TRUE)) %>% RunPCA()

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
ElbowPlot(t_cells, ndims = 50)

In [ ]:
t_cells <- RunUMAP(t_cells, dims=1:40) %>% FindNeighbors(dims=1:40) %>% FindClusters(resolution=0.5)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
DimPlot(t_cells, label = TRUE, repel = TRUE)
DimPlot(t_cells, group.by = 'predicted.celltype.l3')
DimPlot(t_cells, group.by = 'batch_id')
DimPlot(t_cells, group.by = 'age', shuffle = TRUE)

# Save

In [ ]:
saveRDS(t_cells, file = 'aging_t_cells.rds')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)
FeaturePlot(t_cells, features = c('RTKN2','TRGC2','ZNF683','KLRC2'))